In [ ]:
import sys
sys.path

In [ ]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize

class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf
    
short_pos = open("positive.txt","r").read()
short_neg = open("negative.txt","r").read()

# move this up here
all_words = []
documents = []


#  j is adject, r is adverb, and v is verb
#allowed_word_types = ["J","R","V"]
allowed_word_types = ["J"]

for p in short_pos.split('\n'):
    documents.append( (p, "pos") )
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

    
for p in short_neg.split('\n'):
    documents.append( (p, "neg") )
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())



save_documents = open("pickled_algos/documents.pickle","wb")
pickle.dump(documents, save_documents)
save_documents.close()


all_words = nltk.FreqDist(all_words)


word_features = list(all_words.keys())[:5000]


save_word_features = open("pickled_algos/word_features5k.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()
#pickle.dump(classifier, save_classifier)
#save_classifier.close()


def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

featuresets = [(find_features(rev), category) for (rev, category) in documents]

random.shuffle(featuresets)
print(len(featuresets))

testing_set = featuresets[10000:]
training_set = featuresets[:10000]


classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

###############
#save_classifier = save_documents

save_classifier = open("pickled_algos/originalnaivebayes5k.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

save_classifier = open("pickled_algos/MNB_classifier5k.pickle","wb")
pickle.dump(MNB_classifier, save_classifier)
save_classifier.close()

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

save_classifier = open("pickled_algos/BernoulliNB_classifier5k.pickle","wb")
pickle.dump(BernoulliNB_classifier, save_classifier)
save_classifier.close()

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

save_classifier = open("pickled_algos/LogisticRegression_classifier5k.pickle","wb")
pickle.dump(LogisticRegression_classifier, save_classifier)
save_classifier.close()


LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

save_classifier = open("pickled_algos/LinearSVC_classifier5k.pickle","wb")
pickle.dump(LinearSVC_classifier, save_classifier)
save_classifier.close()


##NuSVC_classifier = SklearnClassifier(NuSVC())
##NuSVC_classifier.train(training_set)
##print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)


SGDC_classifier = SklearnClassifier(SGDClassifier())
SGDC_classifier.train(training_set)
print("SGDClassifier accuracy percent:",nltk.classify.accuracy(SGDC_classifier, testing_set)*100)

save_classifier = open("pickled_algos/SGDC_classifier5k.pickle","wb")
pickle.dump(SGDC_classifier, save_classifier)
save_classifier.close()

In [2]:
#this is a test
import sentiment_mod as s

print(s.sentiment("This movie was awesome! The acting was great, plot was wonderful, and there were pythons...so yea!"))
print(s.sentiment("This movie was utter junk. There were absolutely 0 pythons. I don't see what the point was at all. Horrible movie, 0/10"))

10664
('pos', 1.0)
('neg', 1.0)


In [6]:
!pip install tweepy

  Using cached https://files.pythonhosted.org/packages/05/f1/2e8c7b202dd04117a378ac0c55cc7dafa80280ebd7f692f1fa8f27fd6288/tweepy-3.6.0-py2.py3-none-any.whl


In [6]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import sentiment_mod as s

#consumer key, consumer secret, access token, access secret.
ckey="PsEoKS2IsbrlWoGJSYnljMqjj"
csecret="xWncKPqSNeXC04t7NuQSCMJV0rGMFeCAsefm6K2fHVShsppJqF"
atoken="877968762523209728-IxAeEEkUvd9jksqwmsczfcb7jbTSI0v"
asecret="NT5S8QQaTsldO7CR4yB4DcNCQVrIT19RCYBCLRcxVJn0K"

class listener(StreamListener):

    def on_data(self, data):

        all_data = json.loads(data)

        tweet = all_data["text"]
        sentiment_value, confidence = s.sentiment(tweet)
        print(tweet, sentiment_value, confidence)

        if confidence*100 >= 80:
            output = open("twitter-out.txt","a")
            output.write(sentiment_value)
            output.write('\n')
            output.close()

        return True

    def on_error(self, status):
        print(status)

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["detective pikachu"])

RT @krishadraws: Detective Pikachu https://t.co/JsNOzHNEsB neg 1.0
RT @adam22: Asked my 4 year old nephew if he would see detective pikachu with me and he said fuck you old man pos 1.0
POKÉMON Detective Pikachu - Official Trailer #1 https://t.co/ah55RsmalH neg 1.0
who woke up one day and thought detective pikachu was a good idea pos 1.0
RT @Chainbody: detective pikachu looks crazy https://t.co/4QPAS5dhAz pos 1.0
RT @Dan_Brenkert: Me: Not super interested in Pokémon Let’s Go 

Also me on the way to see Detective Pikachu https://t.co/eQhaICKaua neg 1.0
RT @Kotaku: "Realistic Pokemon" artist got a job on the Detective Pikachu movie. https://t.co/b0xl8SXzuu https://t.co/TxcC1xxwxh neg 1.0
RT @jamieloftusHELP: if i don't get fingered to orgasm in a packed screening of 'detective pikachu' so help me neg 0.6
RT @housewaifu: Say what you want about Detective Pikachu but this influx of fluffy Pikachu fanart is easily one of the best things all yea… neg 0.6
RT @griffindorrice: This was my only r

Detective pikachu getting wild neg 1.0
RT @adam22: Asked my 4 year old nephew if he would see detective pikachu with me and he said fuck you old man pos 1.0
RT @griffindorrice: This was my only response to that Detective Pikachu trailer https://t.co/2mSUrRWnTX neg 1.0
RT @adam22: Asked my 4 year old nephew if he would see detective pikachu with me and he said fuck you old man pos 1.0
RT @nutz_teera: ถ้ามิวสิคพากย์ Detective Pikachu คงประมาณนี้ คือดีย์อะ! (ﾉ≧∇≦)ﾉ ﾐ กรี๊ด~

ปล. เสียงทั้งหมดมาจากโคโคทามะ เสียง"ฮื้อ" ตอนโผล่… neg 1.0
RT @ambaex_: When I go see Detective Pikachu by myself https://t.co/dRBByXTNId neg 1.0
I liked a @YouTube video https://t.co/xtrGylehWk Detective Pikachu ANALYSIS - Reveal Trailer (Secrets &amp; All Pokémon Identified!) neg 1.0
RT @fyeahnix: I know this is a long shot but this is why artists need a social media presence and portfolio. People can and DO get jobs for… neg 1.0
RT @nutz_teera: ถ้ามิวสิคพากย์ Detective Pikachu คงประมาณนี้ คือดีย์อะ! (ﾉ≧∇≦)ﾉ ﾐ กรี๊ด

RT @adampknave: I just wanna know who called it Detective Pikachu when they could've called it P.I. Kachu. neg 1.0
RT @RoosterTeeth: Detective Pikachu is gonna get to the bottom of Austin airpot construction. https://t.co/ePvN6Ndgsk neg 1.0
I liked a @YouTube video https://t.co/rO8oxLguOb "POKÉMON Detective Pikachu" Official Trailer #1 REACTION!!!! neg 1.0
I’ve been thinking about it and I have decided I love Detective Pikachu. That is all. pos 1.0
Gostei de um vídeo @YouTube https://t.co/9vw5sHHns2ÉMON Detective Pikachu - Official Trailer #1 pos 0.6
this was the worst tweet i’ve ever had neg 1.0
RT @fyeahnix: I know this is a long shot but this is why artists need a social media presence and portfolio. People can and DO get jobs for… neg 1.0
RT @SethEverman: within minutes you managed to completely ruin detective pikachu https://t.co/WsAQqrgzAh pos 1.0
I want my own Detective Pikachu. Ryan Reynolds voice and all neg 1.0
RT @vincestaples: Finna push to the set and run these fades since

Pokémon: Detective Pikachu | Trailer en Español Latino. https://t.co/mQdTgLHCJj https://t.co/P6TGSzhf4l neg 1.0
RT @DolanDark: If Detective Pikachu was voiced by Alex Jones https://t.co/SIDqAhecOk neg 1.0
RT @harleivy: i've only known detective pikachu for a day but if anything happened to him i would kill everyone in this room and then mysel… pos 0.6
RT @adampknave: I just wanna know who called it Detective Pikachu when they could've called it P.I. Kachu. neg 1.0
RT @prozdkp: detective pikachu https://t.co/CCMgk14vzB pos 1.0
RT @IGNJapan: 【ギャラリー】実写版『名探偵ピカチュウ』で使われたポケモンのモデルと小道具 
https://t.co/XDCIV7DPdB https://t.co/rU3XehZd9Y neg 1.0
RT @DolanDark: If Detective Pikachu was voiced by Alex Jones https://t.co/SIDqAhecOk neg 1.0
RT @maniadrone: Detective Pikachu looks great https://t.co/q34HxgHmGf pos 1.0
RT @harleivy: i've only known detective pikachu for a day but if anything happened to him i would kill everyone in this room and then mysel… pos 0.6
RT @DolanDark: If Detective Pikachu was

RT @griffindorrice: This was my only response to that Detective Pikachu trailer https://t.co/2mSUrRWnTX neg 1.0
RT @DolanDark: If Detective Pikachu was voiced by Alex Jones https://t.co/SIDqAhecOk neg 1.0
RT @NoShape_: Danny Devito is Detective Pikachu in some better timeline, and this is what it sounds like: https://t.co/g029RoGPIh neg 1.0
RT @harleivy: i've only known detective pikachu for a day but if anything happened to him i would kill everyone in this room and then mysel… pos 0.6
Detective Pikachu, Alita: Battle Angel, and Toy Story 4? I'll actually go to the movies in 2019 neg 1.0
RT @fyeahnix: I know this is a long shot but this is why artists need a social media presence and portfolio. People can and DO get jobs for… neg 1.0
RT @jpbrammer: save us... detective pikachu... pos 1.0
RT @adampknave: I just wanna know who called it Detective Pikachu when they could've called it P.I. Kachu. neg 1.0
RT @fyeahnix: I know this is a long shot but this is why artists need a social media 

...

RT @nyarlify: If you're going to watch Detective Pikachu, then don't go to the cinema with your friends who have not read Umineko yet. They… pos 1.0
RT @prozdkp: detective pikachu https://t.co/CCMgk14vzB pos 1.0
RT @fyeahnix: I know this is a long shot but this is why artists need a social media presence and portfolio. People can and DO get jobs for… neg 1.0
Excited for detective pikachu pos 1.0
Detective pikachu has already proven itself to be very divisive in my household as Romey and I fall on opposite sid… https://t.co/BooPnnxOhz neg 1.0
RT @laura_viner: - The Toy Story 4 trailer released
- Ryan Reynolds is voicing Detective Pikachu
- Jennie made a solo debut
- Stan Lee pass… neg 0.6
RT @AlishaRai: BUT JUST TAKE A MINUTE AND IMAGINE Mindy Kaling or Ali Wong as Pikachu detective. Imagine. pos 1.0
RT @adampknave: I just wanna know who called it Detective Pikachu when they could've called it P.I. Kachu. neg 1.0
RT @SethEverman: within minutes you managed to completely ruin detective p

Detective Pikachu could have been voiced by Danny Devito but it's clear that the people at Warner Brothers are COWARDS neg 1.0
RT @natsooee: someone cast seokjin for the korean dub https://t.co/elgT75bVtD pos 1.0
me outside the movie theater opening day for Detective Pikachu https://t.co/yIYrI5z4jX pos 1.0
I liked a @YouTube video https://t.co/DLRn6Jlpcu POKEMON Detective Pikachu Trailer Español Latino (2019) neg 1.0
RT @DolanDark: If Detective Pikachu was voiced by Alex Jones https://t.co/SIDqAhecOk neg 1.0
RT @NoShape_: Danny Devito is Detective Pikachu in some better timeline, and this is what it sounds like: https://t.co/g029RoGPIh neg 1.0
RT @OkamiWolven: heres my opinion on the detective pikachu designs:

they work with the style of movie very well and im super in love. supe… pos 1.0
RT @adampknave: I just wanna know who called it Detective Pikachu when they could've called it P.I. Kachu. neg 1.0
RT @laura_viner: - The Toy Story 4 trailer released
- Ryan Reynolds is voicing Detec

RT @DolanDark: If Detective Pikachu was voiced by Alex Jones https://t.co/SIDqAhecOk neg 1.0
RT @DolanDark: If Detective Pikachu was voiced by Alex Jones https://t.co/SIDqAhecOk neg 1.0
The new ‘Detective Pikachu’ movie trailer features cute and creepy ‘real-life’ versions of Pokémon — here they all … https://t.co/kOZw2sx0X0 neg 1.0
RT @kahonoii: หลายคนสงสัยว่า "ทำไมปิกะจู ใน #DetectivePikachu ถึงเสียงเข้มแบบนั้น?"

หนังดัดแปลงมาจากเกมในชื่อ Detective Pikachu (名探偵ピカチュウ)… neg 1.0
RT @immolations: everyone on this website is like "all cops are bastards!" but all it takes is for one pikachu to become a detective and su… pos 1.0
RT @prozdkp: detective pikachu https://t.co/CCMgk14vzB pos 1.0
RT @NoShape_: Danny Devito is Detective Pikachu in some better timeline, and this is what it sounds like: https://t.co/g029RoGPIh neg 1.0
RT @Kotaku: "Realistic Pokemon" artist got a job on the Detective Pikachu movie. https://t.co/b0xl8SXzuu https://t.co/TxcC1xxwxh neg 1.0
RT @harleivy: i've only known

RT @sonat_isik: tüm şerefimizi haysiyetimizi bi kenara bırakıp detective pikachu filmini 7 yaşında çocuklarla beraber oturup izleyeceğiz bi… pos 1.0
RT @adampknave: I just wanna know who called it Detective Pikachu when they could've called it P.I. Kachu. neg 1.0
Me ha gustado un vídeo de @YouTube (https://t.co/1niDKYoD15 - Pokémon Detective Pikachu Trailer Oficial Español Latino). pos 0.6
RT @prozdkp: detective pikachu https://t.co/CCMgk14vzB pos 1.0
POKÉMON Detective Pikachu - Official Trailer #1 https://t.co/6OqrP98Aw1 @YouTube 님이 공유
헐리우드 실사영화 명탐정 피카츄 예고편. 피카츄 목… https://t.co/hJiPcC1Y2O neg 1.0
justice smith in detective pikachu 😭😍 pos 1.0
Y bien, luego del tráiler de detective pikachu tengo un poco más de esperanza con el live action de bnha pos 1.0
ESTOY EXCITADA neg 1.0
@monibazz No me digas que te gustó el Pikachu detective pos 1.0
RT @adampknave: I just wanna know who called it Detective Pikachu when they could've called it P.I. Kachu. neg 1.0
RT @fyeahnix: I know this is a lon

KeyboardInterrupt: 

In [70]:
#twitter-out.txt
with open('twitter-out.txt', 'r') as myfile:
    data=myfile.read().replace('\n', ',')

print("Sentiment for 'detective pikachu'")
neg = data.count("neg")
print("Negative Sentiment:","\n",neg,"\n",round(neg/total*100,2),"%")
pos = data.count("pos")
print("Positive Sentiment:","\n",pos,"\n",round(pos/total*100,2),"%")
total = pos + neg
print("Total:",total)

Sentiment for 'detective pikachu'
Negative Sentiment: 
 1062 
 64.76 %
Positive Sentiment: 
 578 
 35.24 %
Total: 1640
